# Container Log Parsing and Analysis for Self-Healing Platform

## Overview
This notebook demonstrates how to collect, parse, and analyze container logs from OpenShift for anomaly detection and troubleshooting. It includes structured log parsing (JSON), pattern recognition, and integration with the self-healing coordination engine.

## Prerequisites
- Access to OpenShift cluster with log collection permissions
- Kubernetes Python client installed
- Running coordination engine in the cluster
- Persistent storage for log data

## Expected Outcomes
- Understand container log collection from OpenShift
- Implement structured log parsing for JSON and text formats
- Identify error patterns and anomalies in application logs
- Extract actionable insights for self-healing workflows

## References
- ADR-012: Notebook Architecture for End-to-End Workflows
- ADR-013: Data Collection and Preprocessing Workflows
- OpenShift Logging Documentation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import re
import warnings
import sys
import os
from collections import Counter, defaultdict
from typing import Dict, List, Optional, Any
from pathlib import Path

# Kubernetes client
try:
    from kubernetes import client, config
    k8s_available = True
    print("✅ Kubernetes client available")
except ImportError:
    k8s_available = False
    print("⚠️ Kubernetes client not available - using simulation mode")

# Setup path for utils module - works from any directory
# Find the notebooks directory by looking for known structure
def find_utils_path():
    """Find utils path regardless of current working directory"""
    # Try multiple possible locations
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    
    # Fallback: search upward from cwd
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

# Try to import common functions, with fallback
try:
    from common_functions import (
        setup_environment, print_environment_info,
        save_processed_data, load_processed_data,
        validate_data_quality
    )
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    print("   Using minimal fallback implementations")
    
    # Minimal fallback implementations
    def setup_environment():
        return {
            'data_dir': '/opt/app-root/src/data',
            'models_dir': '/opt/app-root/src/models',
            'working_dir': os.getcwd()
        }
    
    def print_environment_info(env_info):
        print(f"📁 Data dir: {env_info.get('data_dir', 'N/A')}")
        print(f"📁 Models dir: {env_info.get('models_dir', 'N/A')}")
    
    def save_processed_data(data, filename):
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        filepath = f'/opt/app-root/src/data/processed/{filename}'
        if filename.endswith('.parquet') and hasattr(data, 'to_parquet'):
            data.to_parquet(filepath)
        elif filename.endswith('.json'):
            with open(filepath, 'w') as f:
                # Handle non-serializable types
                if hasattr(data, 'items'):
                    serializable = {}
                    for k, v in data.items():
                        if hasattr(v, 'to_dict'):
                            serializable[k] = v.to_dict()
                        else:
                            serializable[k] = v
                    json.dump(serializable, f, default=str)
                else:
                    json.dump(data, f, default=str)
        print(f"💾 Saved: {filepath}")
    
    def load_processed_data(filename):
        filepath = f'/opt/app-root/src/data/processed/{filename}'
        if filename.endswith('.parquet'):
            return pd.read_parquet(filepath)
        elif filename.endswith('.json'):
            with open(filepath, 'r') as f:
                return json.load(f)
        return None
    
    def validate_data_quality(df):
        return {'valid': True, 'issues': []}

print("✅ Libraries imported successfully")

## 2. Setup & Configuration

Initialize the Kubernetes client and configure parameters for log collection from container pods.

In [ ]:
# Set up environment
env_info = setup_environment()
print_environment_info(env_info)

# Log Collection Configuration
LOG_CONFIG = {
    'target_namespaces': ['self-healing-platform', 'openshift-monitoring'],
    'target_pods': ['coordination-engine', 'cluster-health-mcp-server'],
    'log_lines_limit': 1000,  # Maximum lines per pod
    'time_window_hours': 24,  # Look back window
    'log_levels': ['ERROR', 'WARN', 'INFO', 'DEBUG'],
    'structured_formats': ['json', 'logfmt'],
    'error_patterns': [
        r'(?i)error|exception|failed|timeout|refused',
        r'(?i)panic|fatal|critical|emergency',
        r'(?i)connection.*(?:refused|timeout|reset)',
        r'(?i)out of memory|oom|memory.*exceeded'
    ]
}

print(f"📋 Log collection configured for {LOG_CONFIG['time_window_hours']} hours")
print(f"🎯 Target namespaces: {', '.join(LOG_CONFIG['target_namespaces'])}")
print(f"🔍 Error patterns: {len(LOG_CONFIG['error_patterns'])} defined")

## 3. Define Log Collection Functions

Define helper functions to collect logs from Kubernetes pods or generate synthetic logs for testing.

In [ ]:
def setup_kubernetes_client():
    """
    Set up Kubernetes client with in-cluster configuration
    """
    try:
        # Try in-cluster config first
        config.load_incluster_config()
        print("✅ Using in-cluster Kubernetes configuration")
    except:
        try:
            # Fallback to local kubeconfig
            config.load_kube_config()
            print("✅ Using local kubeconfig")
        except:
            print("❌ Failed to load Kubernetes configuration")
            return None
    
    return client.CoreV1Api()

def collect_pod_logs(namespace, pod_name, container_name=None, lines=1000):
    """
    Collect logs from a specific pod/container
    
    Args:
        namespace: Kubernetes namespace
        pod_name: Pod name
        container_name: Container name (optional)
        lines: Number of log lines to retrieve
    
    Returns:
        List of log lines with metadata
    """
    if not k8s_available:
        return generate_synthetic_logs(pod_name, lines)
    
    v1 = setup_kubernetes_client()
    if v1 is None:
        return generate_synthetic_logs(pod_name, lines)
    
    try:
        # Get pod logs
        log_response = v1.read_namespaced_pod_log(
            name=pod_name,
            namespace=namespace,
            container=container_name,
            tail_lines=lines,
            timestamps=True
        )
        
        # Parse log lines
        log_entries = []
        for line in log_response.split('\n'):
            if line.strip():
                # Try to extract timestamp if present
                timestamp_match = re.match(r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z)\s+(.*)$', line)
                if timestamp_match:
                    timestamp_str, message = timestamp_match.groups()
                    timestamp = pd.to_datetime(timestamp_str)
                else:
                    timestamp = datetime.now()
                    message = line
                
                log_entries.append({
                    'timestamp': timestamp,
                    'namespace': namespace,
                    'pod_name': pod_name,
                    'container_name': container_name or 'unknown',
                    'message': message,
                    'raw_line': line
                })
        
        print(f"  ✅ Collected {len(log_entries)} log lines from {pod_name}")
        return log_entries
        
    except Exception as e:
        print(f"  ⚠️ Failed to collect logs from {pod_name}: {e}")
        return generate_synthetic_logs(pod_name, lines)

def collect_logs_from_namespace(namespace, config):
    """
    Collect logs from all relevant pods in a namespace
    """
    print(f"📡 Collecting logs from namespace: {namespace}")
    
    if not k8s_available:
        return generate_synthetic_namespace_logs(namespace, config)
    
    v1 = setup_kubernetes_client()
    if v1 is None:
        return generate_synthetic_namespace_logs(namespace, config)
    
    all_logs = []
    
    try:
        # Get all pods in namespace
        pods = v1.list_namespaced_pod(namespace=namespace)
        
        for pod in pods.items:
            pod_name = pod.metadata.name
            
            # Filter by target pods if specified
            if config['target_pods']:
                if not any(target in pod_name for target in config['target_pods']):
                    continue
            
            # Collect logs from each container
            for container in pod.spec.containers:
                container_logs = collect_pod_logs(
                    namespace, pod_name, container.name, config['log_lines_limit']
                )
                all_logs.extend(container_logs)
        
    except Exception as e:
        print(f"  ⚠️ Failed to list pods in {namespace}: {e}")
        return generate_synthetic_namespace_logs(namespace, config)
    
    print(f"  📊 Total logs collected: {len(all_logs)}")
    return all_logs

# Test Kubernetes connection
if k8s_available:
    v1_test = setup_kubernetes_client()
    if v1_test:
        print("🔗 Kubernetes client connection successful")
    else:
        print("⚠️ Kubernetes client connection failed - will use synthetic data")
else:
    print("⚠️ Kubernetes client not available - will use synthetic data")

## 4. Generate Synthetic Logs

Generate realistic synthetic container logs with various log levels and error patterns for analysis.

In [ ]:
def generate_synthetic_logs(pod_name, lines=1000):
    """
    Generate realistic synthetic container logs for testing
    """
    log_patterns = {
        'INFO': [
            'Starting application server on port 8080',
            'Database connection established',
            'Processing request from user {}',
            'Cache hit for key: {}',
            'Health check passed'
        ],
        'WARN': [
            'High memory usage detected: {}%',
            'Slow query detected: {}ms',
            'Connection pool nearly exhausted',
            'Deprecated API endpoint accessed'
        ],
        'ERROR': [
            'Failed to connect to database: connection timeout',
            'Exception in request handler: {}',
            'Authentication failed for user {}',
            'Out of memory error in worker process',
            'Network connection refused'
        ]
    }
    
    log_entries = []
    start_time = datetime.now() - timedelta(hours=24)
    
    for i in range(lines):
        # Weight log levels realistically
        level = np.random.choice(['INFO', 'WARN', 'ERROR'], p=[0.7, 0.2, 0.1])
        pattern = np.random.choice(log_patterns[level])
        
        # Add realistic values to patterns
        if '{}' in pattern:
            if 'user' in pattern:
                value = f"user{np.random.randint(1000, 9999)}"
            elif '%' in pattern:
                value = np.random.randint(70, 95)
            elif 'ms' in pattern:
                value = np.random.randint(1000, 5000)
            else:
                value = f"value{np.random.randint(100, 999)}"
            message = pattern.format(value)
        else:
            message = pattern
        
        timestamp = start_time + timedelta(
            seconds=np.random.uniform(0, 24*3600)
        )
        
        # Create structured log entry
        if np.random.random() < 0.3:  # 30% JSON logs
            json_log = {
                'timestamp': timestamp.isoformat(),
                'level': level,
                'message': message,
                'component': pod_name,
                'thread': f"thread-{np.random.randint(1, 10)}"
            }
            raw_line = json.dumps(json_log)
        else:  # 70% plain text logs
            raw_line = f"{timestamp.isoformat()} [{level}] {message}"
        
        log_entries.append({
            'timestamp': timestamp,
            'namespace': 'synthetic',
            'pod_name': pod_name,
            'container_name': 'main',
            'message': message,
            'raw_line': raw_line,
            'level': level
        })
    
    return log_entries

def generate_synthetic_namespace_logs(namespace, config):
    """
    Generate synthetic logs for an entire namespace
    """
    all_logs = []
    
    # Generate logs for target pods or default pods
    pods = config['target_pods'] if config['target_pods'] else ['app-server', 'worker', 'cache']
    
    for pod_name in pods:
        pod_logs = generate_synthetic_logs(f"{pod_name}-{np.random.randint(1000, 9999)}", 
                                         config['log_lines_limit'] // len(pods))
        # Update namespace
        for log in pod_logs:
            log['namespace'] = namespace
        all_logs.extend(pod_logs)
    
    return all_logs

# Collect logs from target namespaces
print("🚀 Starting log collection...")
all_logs = []

for namespace in LOG_CONFIG['target_namespaces']:
    namespace_logs = collect_logs_from_namespace(namespace, LOG_CONFIG)
    all_logs.extend(namespace_logs)

# Convert to DataFrame
logs_df = pd.DataFrame(all_logs)
logs_df['timestamp'] = pd.to_datetime(logs_df['timestamp'], utc=True)
logs_df['timestamp'] = logs_df['timestamp'].dt.tz_localize(None)
logs_df = logs_df.sort_values('timestamp')

print(f"\n📊 Log Collection Summary:")
print(f"Total log entries: {len(logs_df)}")
print(f"Time range: {logs_df['timestamp'].min()} to {logs_df['timestamp'].max()}")
print(f"Namespaces: {logs_df['namespace'].nunique()}")
print(f"Unique pods: {logs_df['pod_name'].nunique()}")

## 5. Parse and Analyze Logs

Parse structured logs, extract error patterns, and identify anomalies in application behavior.

In [ ]:
def parse_structured_logs(logs_df):
    """
    Parse structured logs (JSON, logfmt) and extract structured data
    """
    print("🔍 Parsing structured logs...")
    
    parsed_logs = []
    
    for idx, row in logs_df.iterrows():
        log_entry = row.to_dict()
        raw_line = row['raw_line']
        
        # Try to parse as JSON
        try:
            json_data = json.loads(raw_line)
            log_entry.update({
                'format': 'json',
                'level': json_data.get('level', 'INFO'),
                'component': json_data.get('component', 'unknown'),
                'thread': json_data.get('thread', 'main'),
                'structured': True
            })
            if 'message' not in log_entry or not log_entry['message']:
                log_entry['message'] = json_data.get('message', raw_line)
        except (json.JSONDecodeError, TypeError):
            # Try to extract log level from plain text
            level_match = re.search(r'\[(DEBUG|INFO|WARN|ERROR|FATAL)\]', raw_line, re.IGNORECASE)
            if level_match:
                log_entry.update({
                    'format': 'text',
                    'level': level_match.group(1).upper(),
                    'structured': False
                })
            else:
                log_entry.update({
                    'format': 'text',
                    'level': 'INFO',
                    'structured': False
                })
        
        parsed_logs.append(log_entry)
    
    parsed_df = pd.DataFrame(parsed_logs)
    print(f"✅ Parsed {len(parsed_df)} log entries")
    print(f"Structured logs: {parsed_df['structured'].sum()}")
    print(f"Log levels found: {', '.join(parsed_df['level'].unique())}")
    
    return parsed_df

def detect_error_patterns(logs_df, patterns):
    """
    Detect error patterns in log messages
    """
    print("🚨 Detecting error patterns...")
    
    error_matches = []
    
    for idx, row in logs_df.iterrows():
        message = row['message']
        raw_line = row['raw_line']
        
        for pattern_idx, pattern in enumerate(patterns):
            if re.search(pattern, message, re.IGNORECASE) or re.search(pattern, raw_line, re.IGNORECASE):
                error_matches.append({
                    'log_index': idx,
                    'timestamp': row['timestamp'],
                    'namespace': row['namespace'],
                    'pod_name': row['pod_name'],
                    'level': row.get('level', 'UNKNOWN'),
                    'pattern_index': pattern_idx,
                    'pattern': pattern,
                    'matched_text': message,
                    'severity': 'HIGH' if pattern_idx < 2 else 'MEDIUM'
                })
    
    error_df = pd.DataFrame(error_matches)
    print(f"✅ Found {len(error_df)} error pattern matches")
    
    if len(error_df) > 0:
        print(f"High severity errors: {len(error_df[error_df['severity'] == 'HIGH'])}")
        print(f"Most common pattern: {error_df['pattern_index'].mode().iloc[0] if not error_df.empty else 'None'}")
    
    return error_df

def analyze_log_patterns(logs_df):
    """
    Analyze patterns in log data for insights
    """
    print("📊 Analyzing log patterns...")
    
    # Time-based analysis
    logs_df['hour'] = logs_df['timestamp'].dt.hour
    logs_df['day_of_week'] = logs_df['timestamp'].dt.day_name()
    
    patterns = {
        'log_levels': logs_df['level'].value_counts(),
        'pods': logs_df['pod_name'].value_counts(),
        'namespaces': logs_df['namespace'].value_counts(),
        'hourly_distribution': logs_df['hour'].value_counts().sort_index(),
        'daily_distribution': logs_df['day_of_week'].value_counts(),
        'structured_percentage': logs_df['structured'].mean() * 100 if 'structured' in logs_df.columns else 0
    }
    
    # Error rate analysis
    error_logs = logs_df[logs_df['level'].isin(['ERROR', 'FATAL'])]
    patterns['error_rate'] = len(error_logs) / len(logs_df) * 100
    patterns['errors_by_pod'] = error_logs['pod_name'].value_counts()
    
    return patterns

# Parse structured logs
parsed_logs_df = parse_structured_logs(logs_df)

# Detect error patterns
error_matches_df = detect_error_patterns(parsed_logs_df, LOG_CONFIG['error_patterns'])

# Analyze patterns
log_patterns = analyze_log_patterns(parsed_logs_df)

print(f"\n📈 Log Analysis Summary:")
print(f"Error rate: {log_patterns['error_rate']:.2f}%")
print(f"Structured logs: {log_patterns['structured_percentage']:.1f}%")
print(f"Most active pod: {log_patterns['pods'].index[0]} ({log_patterns['pods'].iloc[0]} logs)")
print(f"Error pattern matches: {len(error_matches_df)}")

## 6. Visualize Log Analysis Results

Create visualizations of log patterns, error frequencies, and anomalies for better understanding of application health.

In [ ]:
# Visualize log analysis results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Container Log Analysis Dashboard', fontsize=16, fontweight='bold')

# Log levels distribution
log_patterns['log_levels'].plot(kind='bar', ax=axes[0,0], color=['green', 'orange', 'red', 'darkred'])
axes[0,0].set_title('Log Levels Distribution')
axes[0,0].set_xlabel('Log Level')
axes[0,0].set_ylabel('Count')
axes[0,0].tick_params(axis='x', rotation=45)

# Top pods by log volume
log_patterns['pods'].head(10).plot(kind='barh', ax=axes[0,1])
axes[0,1].set_title('Top 10 Pods by Log Volume')
axes[0,1].set_xlabel('Log Count')

# Hourly log distribution
log_patterns['hourly_distribution'].plot(kind='line', ax=axes[1,0], marker='o')
axes[1,0].set_title('Log Volume by Hour of Day')
axes[1,0].set_xlabel('Hour')
axes[1,0].set_ylabel('Log Count')
axes[1,0].grid(True, alpha=0.3)

# Error patterns over time (if errors found)
if len(error_matches_df) > 0:
    error_matches_df.set_index('timestamp')['severity'].resample('1H').count().plot(
        kind='line', ax=axes[1,1], marker='o', color='red'
    )
    axes[1,1].set_title('Error Patterns Over Time')
    axes[1,1].set_xlabel('Time')
    axes[1,1].set_ylabel('Error Count')
else:
    axes[1,1].text(0.5, 0.5, 'No Error Patterns\nDetected', 
                   ha='center', va='center', transform=axes[1,1].transAxes,
                   fontsize=14, color='green')
    axes[1,1].set_title('Error Patterns Over Time')

plt.tight_layout()
plt.show()

# Save processed log data
save_processed_data(parsed_logs_df, 'container_logs_parsed.parquet')
save_processed_data(error_matches_df, 'log_error_patterns.parquet')
save_processed_data(log_patterns, 'log_analysis_patterns.json')

print("\n💾 Data saved successfully:")
print("- container_logs_parsed.parquet: Parsed log data with structure")
print("- log_error_patterns.parquet: Detected error patterns")
print("- log_analysis_patterns.json: Log pattern analysis results")

## Integration with Coordination Engine

This section demonstrates how to integrate log analysis with the self-healing coordination engine.

In [ ]:
# Import MCP client for coordination engine integration
# Use the same path finding logic as cell-1
try:
    from mcp_client import get_cluster_health_client
    mcp_available = True
    print("✅ MCP client imported")
except ImportError:
    mcp_available = False
    print("⚠️ MCP client not available - using simulation mode")
    
    # Fallback MCP client simulation
    class SimulatedMCPClient:
        def query_anomaly_patterns(self, data):
            return {
                'status': 'simulated',
                'simulated': True,
                'anomalies': [],
                'message': 'MCP client not available - using simulation'
            }
    
    def get_cluster_health_client():
        return SimulatedMCPClient()

def send_log_analysis_to_coordination_engine(logs_df, error_matches_df, patterns):
    """
    Send log analysis results to coordination engine for action
    """
    print("🔗 Integrating with coordination engine...")
    
    # Get Cluster Health MCP client
    mcp_client = get_cluster_health_client()
    
    # Prepare log analysis summary
    log_summary = {
        'timestamp': datetime.now().isoformat(),
        'total_logs': len(logs_df),
        'error_rate': patterns['error_rate'],
        'structured_percentage': patterns['structured_percentage'],
        'error_patterns_found': len(error_matches_df),
        'top_error_pods': patterns['errors_by_pod'].head(5).to_dict() if len(patterns['errors_by_pod']) > 0 else {},
        'log_levels': patterns['log_levels'].to_dict(),
        'anomaly_indicators': {
            'high_error_rate': patterns['error_rate'] > 5.0,
            'critical_errors_present': len(error_matches_df[error_matches_df['severity'] == 'HIGH']) > 0 if len(error_matches_df) > 0 else False,
            'pod_concentration': patterns['pods'].iloc[0] / len(logs_df) > 0.5 if len(patterns['pods']) > 0 else False
        }
    }
    
    # Send to coordination engine
    try:
        response = mcp_client.query_anomaly_patterns({
            'source': 'container-logs',
            'data': log_summary,
            'severity': 'high' if log_summary['anomaly_indicators']['critical_errors_present'] else 'medium'
        })
        
        print(f"✅ Log analysis sent to coordination engine")
        if response.get('simulated'):
            print(f"   (Using simulation mode - MCP server not available)")
        print(f"   Anomalies detected: {len(response.get('anomalies', []))}")
        return response
        
    except Exception as e:
        print(f"⚠️ Failed to send to coordination engine: {e}")
        return {'status': 'failed', 'error': str(e), 'anomalies': []}

def generate_log_based_alerts(error_matches_df, patterns, logs_df):
    """
    Generate actionable alerts based on log analysis
    """
    print("🚨 Generating log-based alerts...")
    
    alerts = []
    
    # High error rate alert
    if patterns['error_rate'] > 5.0:
        alerts.append({
            'type': 'high_error_rate',
            'severity': 'HIGH',
            'message': f"Error rate is {patterns['error_rate']:.2f}%, exceeding 5% threshold",
            'recommended_action': 'Investigate error patterns and consider scaling or restarting affected pods'
        })
    
    # Critical error patterns alert
    if len(error_matches_df) > 0:
        critical_errors = error_matches_df[error_matches_df['severity'] == 'HIGH']
        if len(critical_errors) > 0:
            alerts.append({
                'type': 'critical_error_patterns',
                'severity': 'CRITICAL',
                'message': f"Found {len(critical_errors)} critical error patterns",
                'affected_pods': critical_errors['pod_name'].unique().tolist(),
                'recommended_action': 'Immediate investigation required for affected pods'
            })
    
    # Pod concentration alert
    if len(patterns['pods']) > 0 and len(logs_df) > 0:
        if patterns['pods'].iloc[0] / len(logs_df) > 0.5:
            alerts.append({
                'type': 'log_concentration',
                'severity': 'MEDIUM',
                'message': f"Pod {patterns['pods'].index[0]} generating {patterns['pods'].iloc[0]} logs (>50% of total)",
                'recommended_action': 'Check if pod is experiencing issues or needs log level adjustment'
            })
    
    print(f"✅ Generated {len(alerts)} alerts")
    return alerts

# Send analysis to coordination engine
coordination_response = send_log_analysis_to_coordination_engine(parsed_logs_df, error_matches_df, log_patterns)

# Generate alerts (pass logs_df to avoid NameError)
alerts = generate_log_based_alerts(error_matches_df, log_patterns, parsed_logs_df)

if alerts:
    print("\n🚨 Generated Alerts:")
    for alert in alerts:
        print(f"- [{alert['severity']}] {alert['type']}: {alert['message']}")
        print(f"  Action: {alert['recommended_action']}")
else:
    print("\n✅ No alerts generated - logs appear healthy")

print("\n🎯 Next Steps:")
print("1. Set up real-time log streaming and analysis")
print("2. Configure alerting thresholds based on baseline")
print("3. Implement automated remediation for common patterns")
print("4. Integrate with anomaly detection models")
print("\n📚 Related Notebooks:")
print("- 02-anomaly-detection/: Use log patterns for ML model training")
print("- 03-self-healing-logic/: Implement log-driven remediation")